# Fashion Classification

### Importing Neccessary Libraries

In [14]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [15]:
tensorflow.__version__

'2.3.0'

In [16]:
tensorflow.keras.__version__

'2.4.0'

### Image Data Agumentation

In [17]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [18]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('dataset/train',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('dataset/test',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 3068 images belonging to 10 classes.
Found 372 images belonging to 10 classes.


In [19]:
print(x_train.class_indices)#checking the number of classes

{'dress': 0, 'hat': 1, 'longsleeve': 2, 'outwear': 3, 'pants': 4, 'shirt': 5, 'shoes': 6, 'shorts': 7, 'skirt': 8, 't-shirt': 9}


In [20]:
print(x_test.class_indices)#checking the number of classes

{'dress': 0, 'hat': 1, 'longsleeve': 2, 'outwear': 3, 'pants': 4, 'shirt': 5, 'shoes': 6, 'shorts': 7, 'skirt': 8, 't-shirt': 9}


In [21]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 241,
         1: 123,
         2: 455,
         3: 184,
         4: 468,
         5: 290,
         6: 198,
         7: 202,
         8: 112,
         9: 795})

### Creating the model

In [22]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=10, activation='softmax')) # softmax for more than 2



In [23]:
classifier.summary()#summary of our model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               802944    
_________________________________________________________________
dense_3 (Dense)              (None, 10)               

### Compiling the model

In [24]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [25]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

Epoch 1/20
614/614 [==============================] - 91s 148ms/step - loss: 1.7736 - accuracy: 0.4087 - val_loss: 1.6338 - val_accuracy: 0.4651
Epoch 2/20
614/614 [==============================] - 100s 163ms/step - loss: 1.2807 - accuracy: 0.5841 - val_loss: 1.1994 - val_accuracy: 0.6317
Epoch 3/20
614/614 [==============================] - 104s 170ms/step - loss: 1.0766 - accuracy: 0.6320 - val_loss: 1.0521 - val_accuracy: 0.6478
Epoch 4/20
614/614 [==============================] - 98s 159ms/step - loss: 0.9641 - accuracy: 0.6783 - val_loss: 1.1402 - val_accuracy: 0.6425
Epoch 5/20
614/614 [==============================] - 49s 80ms/step - loss: 0.8423 - accuracy: 0.7099 - val_loss: 1.0469 - val_accuracy: 0.6613
Epoch 6/20
614/614 [==============================] - 77s 126ms/step - loss: 0.7884 - accuracy: 0.7347 - val_loss: 1.1024 - val_accuracy: 0.6694
Epoch 7/20
614/614 [==============================] - 62s 101ms/step - loss: 0.7207 - accuracy: 0.7546 - val_loss: 1.0333 - val_a

### Saving our model

In [26]:
# Save the model
classifier.save('fashion.h5')

### Predicting our results

In [27]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("fashion.h5") #loading the model for testing

In [28]:
img = image.load_img(r"D:\SmartBridge\IBM_Hack_Challenge 2021\FashionBot\dataset\train\shirt\0d293c69-ba4f-4ffc-b6b9-fa05316a02c5.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict_classes(x)#predicting the classes
pred

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([5], dtype=int64)

In [30]:
index=['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt', 'shoes', 'shorts', 'skirt', 't-shirt']
result=str(index[pred[0]])
result

'shirt'